In [7]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns

In [29]:
!pip install BatchNormalization

ERROR: Could not find a version that satisfies the requirement BatchNormalization (from versions: none)
ERROR: No matching distribution found for BatchNormalization


In [30]:
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization

In [10]:
print(os.listdir("data"))

['Test', 'Train']


In [11]:
SIZE = 128

In [14]:
train_images = []
train_labels = []
for directory_path in glob.glob("data/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

1-s2.0-S1319562X12000332-gr5 - Copy.jpg
1-s2.0-S1319562X12000332-gr5.jpg
15-investigatin - Copy.jpg
15-investigatin.jpg
315748_1_En_8_Fig2_HTML - Copy.jpg
315748_1_En_8_Fig2_HTML.jpg
aam-aaes80642 - Copy.jpg
aam-aaes80642.jpg
algae-for-water-treatment-system-2-320 - Copy.jpg
algae-for-water-treatment-system-2-320.jpg
Algae-typically-found-in-facultative-ponds-a-Chlamydomonas-b-Chlorella-c_Q320 - Copy.jpg
Algae-typically-found-in-facultative-ponds-a-Chlamydomonas-b-Chlorella-c_Q320.jpg
Australian-algae-producer-enters-animal-feed-market - Copy.jpg
Australian-algae-producer-enters-animal-feed-market.jpg
d-various-rod-shaped-bacteria-magnification-2000x-at-35mm-image-size-2BE0GTE - Copy.jpg
d-various-rod-shaped-bacteria-magnification-2000x-at-35mm-image-size-2BE0GTE.jpg
eg3 - Copy.jpg
eg3.jpg
euglena - Copy.jpg
euglena-2GRMTJ2 - Copy.jpg
euglena-2GRMTJ2.jpg
euglena-dsc_0169 - Copy.jpg
euglena-dsc_0169.jpg
EUGLENA-labels-600 - Copy.jpg
EUGLENA-labels-600.jpg
euglena-with-flagella-captioned

In [15]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [16]:
# test
test_images = []
test_labels = []
for directory_path in glob.glob("data/test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

In [17]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [18]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [23]:
!pip install keras.utils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2634 sha256=f635fed9fe911c53964bc51a721bb28973b082c70b965dbcd5069e3aff48bedc
  Stored in directory: c:\users\kiit\appdata\local\pip\cache\wheels\5c\c0\b3\0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


In [32]:
# Normalize pixel values to between 0 and 1
#x_train, x_test = x_train / 255.0, x_test / 255.0

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [ ]:
import sys
def process_image():
    min = sys.maxsize
    max = -sys.maxsize
    image = Image.open(data)
    new_image = image.resize((224,224))    
    np_image = asarray(image)
    if min > np_image.min():
        min = np_image.min()
    if max < np_image.max():
        max = np_image.max()    

    np_image = np_image.astype('float32')
    print("BEGINNING PIXEL VALUES", np_image)
    print('Data Type: %s' % np_image.dtype)
    print('Min: %.3f, Max: %.3f' % (np_image.min(), np_image.max()))
    np_image -= min
    np_image /= (max - min)

    print('Min: %.3f, Max: %.3f' % (np_image.min(), np_image.max()))
    print(np_image)

In [31]:
#One hot encode y values for neural network.
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

ValueError: zero-size array to reduction operation maximum which has no identity